In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv


In [2]:
import contractions as cont
from num2words import num2words

ModuleNotFoundError: No module named 'contractions'

In [3]:
dataset = pd.read_csv('/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [4]:
dataset.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [5]:
cont.fix("i'm")

NameError: name 'cont' is not defined

In [6]:
def expand_contraction(x):
    return cont.fix(x)

In [7]:
def treat_num(x):
    temp = x.split()
    sentence = []
    for i in temp:
        try:
            word = num2words(i)
            sentence.append(word)
        except:
            word = i
            sentence.append(word)
    return ' '.join(sentence)

In [8]:
from nltk.tokenize import RegexpTokenizer as ret

In [9]:
def rm_symbol(x):
    tokenizer = ret(r'\w+')
    result_with_underscore = tokenizer.tokenize(text=x)
    result_with_underscore = ' '.join(result_with_underscore)
    tokenizer = ret(r'[^_]')
    res = tokenizer.tokenize(result_with_underscore)
    res = ''.join(res)
    return res

In [10]:
from nltk.stem import WordNetLemmatizer 

In [11]:
def lemmatize(x):
    temp = x.split()
    lem = WordNetLemmatizer()
    sentence=[]
    for i in temp:
        sentence.append(lem.lemmatize(i))
    return ' '.join(sentence)

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [13]:
def rm_stopword(x):
    stopword = set(stopwords.words('english'))
    token = word_tokenize(x)
    sentence = [i for i in token if not i in stopword]
    return ' '.join(sentence)

In [14]:
def get_cleaned(x):
#     print(x)
    x = expand_contraction(x)
    x = treat_num(x)
    x = rm_symbol(x)
    x = rm_stopword(x)
    x = lemmatize(x)  
    return x

In [15]:
dataset.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
dataset['message'] = dataset.Message.apply(get_cleaned)

NameError: name 'cont' is not defined

In [17]:
dataset.drop('Message', inplace=True, axis=1)

In [18]:
dataset.head()

,Category
0,ham
1,ham
2,spam
3,ham
4,ham


In [19]:
import tensorflow as tf
from keras import Sequential
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
x = dataset.message
y = dataset.Category
y = pd.get_dummies(y)

AttributeError: 'DataFrame' object has no attribute 'message'

In [23]:
trainx, testx, trainy, testy = train_test_split(x, y, test_size=0.2)

NameError: name 'x' is not defined

# Model using Sklearn tokenizer

In [24]:
vect = TfidfVectorizer()
vect.fit(trainx)
def get_vect(x):
    return vect.transform(x)
train_data = vect.transform(trainx)
test_data = vect.transform(testx)

NameError: name 'trainx' is not defined

In [25]:
train_data

NameError: name 'train_data' is not defined

In [26]:
len(vect.vocabulary_)

AttributeError: 'TfidfVectorizer' object has no attribute 'vocabulary_'

In [27]:
model = Sequential([
    Embedding(6962, 64),
    Bidirectional(LSTM(64, dropout=0.7, return_sequences=True)),  #return_sequences because we are feeding it to next LSTM
    LSTM(64, dropout=0.7),
    Dense(2, activation='softmax'),
])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          445568    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         66048     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 561,154
Trainable params: 561,154
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

model.fit(train_data, trainy.values, epochs=5, validation_split=0.2, batch_size=64)

model.evaluate(test_data, testy.values)

In [30]:
# This cannot be executed because of very high time requirement 

# Model Using DL Tokenizer 

In [31]:
vocab_size = 1000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(trainx.values)
trainseq = tokenizer.texts_to_sequences(trainx.values)
testseq = tokenizer.texts_to_sequences(testx.values)
trainpad = pad_sequences(trainseq, 25)
testpad = pad_sequences(testseq, 25)

NameError: name 'trainx' is not defined

In [32]:
len(tokenizer.word_index)

0

In [33]:
trainpad.shape

NameError: name 'trainpad' is not defined

In [34]:
(trainpad[0])

NameError: name 'trainpad' is not defined

In [35]:
tokenizer.sequences_to_texts(trainpad)

NameError: name 'trainpad' is not defined

In [36]:
def get_dl_token(x):
    x = tokenizer.texts_to_sequences(x)
    x = pad_sequences(x, 50)
    return x

In [37]:
model = Sequential([
    Embedding(6962, 64),
    Bidirectional(LSTM(64, dropout=0.7, return_sequences=True)),  #return_sequences because we are feeding it to next LSTM
    Bidirectional(LSTM(64, dropout=0.7)),  #https://www.tensorflow.org/tutorials/text/text_classification_rnn#stack_two_or_more_lstm_layers
    Dense(2, activation='sigmoid'),
])

In [38]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [39]:
history = model.fit(trainpad, trainy.values, epochs=5, validation_split=0.2, batch_size=64)

NameError: name 'trainpad' is not defined

In [40]:
history.history.keys()

NameError: name 'history' is not defined

In [41]:
import matplotlib.pyplot as plt

In [42]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend(['val_loss', 'loss'])

NameError: name 'history' is not defined

In [43]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])

NameError: name 'history' is not defined

In [44]:
model.evaluate(testpad, testy.values)

NameError: name 'testpad' is not defined

In [45]:
tokenizer.get_config()

{'num_words': 1000,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 0,
 'word_counts': '{}',
 'word_docs': '{}',
 'index_docs': '{}',
 'index_word': '{}',
 'word_index': '{}'}

In [46]:
def get_pred(x):
    pred = model.predict(x)
    for i, j in pred:
        if i>0.5:
            print('Ham')
        else:
            print('spam')

In [47]:
def pipeline(x):
    temp = pd.Series(x)
    for x in temp:
        x = get_cleaned(x)
        x = get_dl_token([x])
        get_pred(x)

In [48]:
pipeline('WINNER As valued network customer selected receivea 900 prize reward To claim call nine billion sixty one million seven hundred one thousand four hundred sixty one Claim code KL341 Valid twelve hour')

NameError: name 'cont' is not defined